<a href="https://colab.research.google.com/github/recon48/20242R0136COSE47402/blob/main/preprocessing_oxfordpets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preparation

In [ ]:
!pip install ultralytics

from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt

import os
from PIL import Image

# COCO 클래스 매핑
category_to_pet_label = {
    # Dog breeds
    "Abyssinian": "cat",
    "American_Bulldog": "dog",
    "American_Pit_Bull_Terrier": "dog",
    "Basset_Hound": "dog",
    "Beagle": "dog",
    "Bengal": "cat",
    "Birman": "cat",
    "Bombay": "cat",
    "Boxer": "dog",
    "British_Shorthair": "cat",
    "Chihuahua": "dog",
    "Egyptian_Mau": "cat",
    "English_Cocker_Spaniel": "dog",
    "English_Setter": "dog",
    "German_Shorthaired": "dog",
    "Great_Pyrenees": "dog",
    "Havanese": "dog",
    "Japanese_Chin": "dog",
    "Keeshond": "dog",
    "Leonberger": "dog",
    "Maine_Coon": "cat",
    "Miniature_Pinscher": "dog",
    "Newfoundland": "dog",
    "Persian": "cat",
    "Pomeranian": "dog",
    "Pug": "dog",
    "Ragdoll": "cat",
    "Russian_Blue": "cat",
    "Saint_Bernard": "dog",
    "Samoyed": "dog",
    "Scottish_Terrier": "dog",
    "Shiba_Inu": "dog",
    "Siamese": "cat",
    "Sphynx": "cat",
    "Staffordshire_Bull_Terrier": "dog",
    "Wheaten_Terrier": "dog",
    "Yorkshire_Terrier": "dog",
}


# 타겟 카테고리 지정
# target_categories = [
#     "airplanes", "car_side", "cellphone", "chair", "cup",
#     "elephant", "pizza", "scissors", "stop_sign", "umbrella", "laptop"
# ]


## Dataset

In [ ]:
!git clone https://github.com/mlvlab/ProMetaR.git
%cd ProMetaR/

!git clone https://github.com/KaiyangZhou/Dassl.pytorch.git
%cd Dassl.pytorch/

# Install dependencies
!pip install -r requirements.txt
!cp -r dassl ../
# Install this library (no need to re-build if the source code is modified)
# !python setup.py develop
%cd ..

!pip install -r requirements.txt

%mkdir outputs
%mkdir data

# %cd data
# %mkdir eurosat
# !wget http://madm.dfki.de/files/sentinel/EuroSAT.zip -O EuroSAT.zip

# !unzip -o EuroSAT.zip -d eurosat/
# %cd eurosat
# !gdown 1Ip7yaCWFi0eaOFUGga0lUdVi_DDQth1o

# %cd ../../
%cd data
%mkdir oxford_pets
!cp "/content/drive/My Drive/images.tar.zip" ./  # Colab의 현재 디렉토리로 복사
# import tarfile

# # 압축 해제
# with tarfile.open('101_ObjectCategories.tar.gz', 'r:gz') as tar:
#     tar.extractall(path='./caltech-101')
!unzip ./images.tar.zip -d ./oxford_pets/

#!unzip -o caltech.zip -d caltech101/
%cd oxford_pets
!cp "/content/drive/My Drive/annotations.tar"

%cd ../../
!pwd
!cp "/content/drive/My Drive/split_zhou_OxfordPets.json" ./data/oxford_pets/
!cp "/content/drive/My Drive/annotations.tar" ./data/oxford_pets/
!tar -xf "./data/oxford_pets/annotations.tar" -C ./data/oxford_pets/
!tar -xf "./data/oxford_pets/images.tar" -C ./data/oxford_pets/

fatal: destination path 'ProMetaR' already exists and is not an empty directory.
/content/ProMetaR
fatal: destination path 'Dassl.pytorch' already exists and is not an empty directory.
/content/ProMetaR/Dassl.pytorch
/content/ProMetaR
mkdir: cannot create directory ‘outputs’: File exists
mkdir: cannot create directory ‘data’: File exists
/content/ProMetaR/data
mkdir: cannot create directory ‘oxford_pets’: File exists
Archive:  ./images.tar.zip
  inflating: ./oxford_pets/images.tar  
  inflating: ./oxford_pets/__MACOSX/._images.tar  
/content/ProMetaR/data/oxford_pets
cp: missing destination file operand after '/content/drive/My Drive/annotations.tar'
Try 'cp --help' for more information.
/content/ProMetaR
/content/ProMetaR


## Detect and Crop

In [ ]:
def detect_and_crop_yolo(image_path, model_path='yolov8n.pt'):
    """
    YOLO를 사용해 이미지에서 특정 타겟 라벨에 해당하는 객체를 탐지하고 Crop합니다.

    Args:
        image_path (str): 이미지 파일 경로.
        target_label (str): 탐지할 타겟 클래스 이름 (예: 'airplane').
        model_path (str): YOLO 모델 가중치 파일 경로.

    Returns:
        cropped_image (PIL.Image): 탐지된 객체 영역을 Crop한 이미지 (없으면 None 반환).
    """
    # 모델 로드
    model = YOLO(model_path)

    # 이미지 열기
    img = Image.open(image_path).convert("RGB")

    # 객체 탐지 수행
    results = model(img)

    # 바운딩 박스 및 클래스 정보 가져오기
    boxes = results[0].boxes.xyxy  # [x_min, y_min, x_max, y_max]
    confidences = results[0].boxes.conf  # 신뢰도
    classes = results[0].boxes.cls  # 클래스 인덱스

    if len(boxes) == 0:
        print(f"No objects detected in {image_path}.")
        return None

    # 타겟 클래스 탐지
    model_classes = model.names  # COCO 클래스 이름
    for idx, box in enumerate(boxes):
        class_name = model_classes[int(classes[idx])]
        if class_name == 'dog' or class_name == 'cat':
            # 타겟 클래스의 바운딩 박스 가져오기
            x_min, y_min, x_max, y_max = map(int, box.tolist())

            # 이미지 Crop
            cropped_image = img.crop((x_min, y_min, x_max, y_max))
            print(f"Detected and cropped '{class_name}' in {image_path}.")
            return cropped_image

    # 타겟 클래스 탐지 실패
    print(f"No target detected in {image_path}.")
    return None


# cropped_image = detect_and_crop_yolo('/content/ProMetaR/data/oxford_pets/images/Abyssinian_100.jpg')
# if cropped_image:
#     # Crop된 이미지 출력
#     cropped_image.show()
#     plt.imshow(cropped_image)
#     print("yes")
# else:
#   print("none")

Crop and Save

In [9]:
import os
from PIL import Image

def crop_and_save_all_images(input_dir, output_dir):
    """
    input_dir 내 모든 이미지를 Crop하고,
    객체 탐지 실패 시 원본 이미지를 저장합니다.

    Args:
        input_dir (str): 원본 이미지 폴더 경로.
        output_dir (str): Crop된 이미지를 저장할 폴더 경로.
    """
    no_detection_count = 0  # 탐지되지 않은 이미지 수
    total_images = 0       # 전체 이미지 수
    detection_count = 0    # 탐지된 이미지 수

    os.makedirs(output_dir, exist_ok=True)  # 출력 폴더 생성

    for image_file in os.listdir(input_dir):
        total_images += 1
        print(f"진행 완료 이미지 수: {total_images}")
        print(f"No detection 이미지 수: {no_detection_count}")
        print(f"Detection 이미지 수: {detection_count}")

        image_path = os.path.join(input_dir, image_file)

        try:
            # Crop 이미지 생성
            cropped_image = detect_and_crop_yolo(image_path)
            if cropped_image is None:
                # 객체 탐지 실패 시 원본 이미지를 저장
                print(f"No detection for {image_path}. Saving original image.")
                cropped_image = Image.open(image_path).convert("RGB")
                no_detection_count += 1
            else:
                detection_count += 1

            # Crop 또는 원본 이미지를 저장
            save_path = os.path.join(output_dir, image_file)
            cropped_image.save(save_path)
            print(f"Saved image: {save_path}")
        except Exception as e:
            print(f"Error processing {image_path}: {e}")

    # 탐지 실패 통계 출력
    print(f"Total images processed: {total_images}")
    print(f"Number of images with no detection: {no_detection_count}")
    print(f"Number of images with detection: {detection_count}")

# Example Usage
input_dir = "/content/ProMetaR/data/oxford_pets/images"  # 원본 이미지 폴더
output_dir = "/content/drive/MyDrive/cropped_oxford_pets_images"  # Crop된 이미지 저장 폴더

# Crop 및 저장 수행
crop_and_save_all_images(input_dir, output_dir)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Detection 이미지 수: 6437

0: 544x640 1 dog, 334.9ms
Speed: 4.7ms preprocess, 334.9ms inference, 1.8ms postprocess per image at shape (1, 3, 544, 640)
Detected and cropped 'dog' in /content/ProMetaR/data/oxford_pets/images/staffordshire_bull_terrier_8.jpg.
Saved image: /content/drive/MyDrive/cropped_oxford_pets_images/staffordshire_bull_terrier_8.jpg
진행 완료 이미지 수: 6773
No detection 이미지 수: 331
Detection 이미지 수: 6438

0: 640x448 1 cat, 286.3ms
Speed: 3.6ms preprocess, 286.3ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 448)
Detected and cropped 'cat' in /content/ProMetaR/data/oxford_pets/images/Siamese_29.jpg.
Saved image: /content/drive/MyDrive/cropped_oxford_pets_images/Siamese_29.jpg
진행 완료 이미지 수: 6774
No detection 이미지 수: 331
Detection 이미지 수: 6439

0: 640x384 1 cat, 261.0ms
Speed: 3.1ms preprocess, 261.0ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 384)
Detected and cropped 'cat' in /content/ProMetaR/data/oxford_pets/images

In [11]:
!zip -r /content/drive/MyDrive/cropped_oxford_pets_images.zip /content/drive/MyDrive/cropped_oxford_pets_images

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  adding: content/drive/MyDrive/cropped_oxford_pets_images/British_Shorthair_50.jpg (deflated 2%)
  adding: content/drive/MyDrive/cropped_oxford_pets_images/chihuahua_181.jpg (deflated 3%)
  adding: content/drive/MyDrive/cropped_oxford_pets_images/keeshond_90.jpg (deflated 1%)
  adding: content/drive/MyDrive/cropped_oxford_pets_images/pomeranian_187.jpg (deflated 2%)
  adding: content/drive/MyDrive/cropped_oxford_pets_images/scottish_terrier_163.jpg (deflated 1%)
  adding: content/drive/MyDrive/cropped_oxford_pets_images/samoyed_124.jpg (deflated 1%)
  adding: content/drive/MyDrive/cropped_oxford_pets_images/german_shorthaired_134.jpg (deflated 4%)
  adding: content/drive/MyDrive/cropped_oxford_pets_images/chihuahua_33.jpg (deflated 1%)
  adding: content/drive/MyDrive/cropped_oxford_pets_images/great_pyrenees_67.jpg (deflated 1%)
  adding: content/drive/MyDrive/cropped_oxford_pets_images/great_pyrenees_35.jpg (deflated 1%)
  adding: content/drive/MyD